In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# Function to parse each line of the file
def parse_line(line):
    # Split the line by commas for the titles or by spaces for the data
    parts = line.strip().split(',') if ',' in line else line.strip().split()
    return parts

# Specify the names of the files to search for
target_files = ['flow_cut_to_failure.vtk', 'transport_cut_to_failure.vtk']
n = 90

# Arrays to store the read data
h2aq = np.zeros([n+1, 1])
h2saq = np.zeros([n+1, 1])
h2g = np.zeros([n+1, 1])

In [9]:
for i in range(1,n+1):
    # Get the current directory
    current_dir = './../Goe_Base_2/run_' + str(i) + '/'

    # Search for the target files in the current directory
    for file in os.listdir(current_dir):
        if file in target_files:
            print("Failure found in folder:", current_dir)
            break  # Exit the loop if either file is found
        else:
            read_folder = True
            
    if not os.path.exists('./../Goe_Base_2/run_' + str(i) + '/pflotran_h2-012.vtk'):
            print(f"File does not exist: {'./../Goe_Base_2/run_' + str(i) + '/pflotran_h2-012.vtk'}")
            continue
    else:
        file_path_h2 = './../Goe_Base_2/run_' + str(i) + '/pflotran_h2-012.vtk'
        #============
        # Read H2(aq)
        # ============
        # Initialize an empty list to store the numbers
        numbers_list_h2 = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2 = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS Total_H2(aq)' in line:
                    start_reading_h2 = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2 = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2_all = np.array(numbers_list_h2)
        numbers_array_h2 = numbers_array_h2_all[numbers_array_h2_all <= 1.0]
        numbers_array_sat_all = np.array(numbers_list_sat)
        numbers_array_sat = numbers_array_sat_all[numbers_array_h2_all <= 1.0]

        cell_vol_gr = 16*16*20*0.2*1000

        # Display the first few numbers to verify
        cell_h2 = cell_vol_gr*numbers_array_sat*numbers_array_h2
        total_h2 = np.sum(cell_h2)

        # print("Total H2(aq) is " + str(total_h2) + " moles")
        h2aq[i, 0] = total_h2

        # ============
        # Read H2S(aq)
         # ============
        file_path_h2s = './../Goe_Base_2/run_' + str(i) + '/pflotran_h2-012.vtk'
        # Initialize an empty list to store the numbers
        numbers_list_h2s = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2s = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2s, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS H2S(aq)' in line:
                    start_reading_h2s = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2s:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2s = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2s.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2s_all = np.array(numbers_list_h2s)
        numbers_array_h2s = numbers_array_h2s_all[(numbers_array_h2s_all <= 1.0) & (numbers_array_h2s_all >= 2e-8)]
        # numbers_array_sat_all = np.array(numbers_list_sat)
        # numbers_array_sat = numbers_array_sat_all[numbers_array_h2s_all <= 1.0 and numbers_array_h2s_all >= 2e-8]

        # cell_vol_gr = 16*16*20*0.2*1000

        # # Display the first few numbers to verify
        # cell_h2s = cell_vol_gr*numbers_array_sat*numbers_array_h2s
        # total_h2s = np.sum(cell_h2s)

        # print("Total H2S(aq) is: " + str(total_h2s) + " moles")
        h2saq[i, 0] = np.average(numbers_array_h2s) # total_h2s

        # ==========
        # Read H2(g)
        # ==========
        # Define the path to your .dat file
        file_path_h2g = './../Goe_Base_2/run_' + str(i) + '/pflotran_h2-mas.dat'

        # Read the file and parse each line
        with open(file_path_h2g, 'r') as file:
            lines = file.readlines()

        # Parse the column titles
        column_titles = parse_line(lines[0])

        # Initialize a list to hold the parsed data
        data = []

        # Parse each line of data
        for line in lines[1:]:  # Skip the first line with column titles
            parsed_line = parse_line(line)
            if parsed_line:  # Ensure the line is not empty
                data.append(parsed_line)

        # Create a DataFrame from the parsed data
        # The length of the longest row is used to ensure all rows have the same length
        max_length = max(len(row) for row in data)
        df = pd.DataFrame(data, columns=column_titles[:max_length])

        df.columns = df.columns.str.strip('"')
        last_number_global_h2 = df["Global H2(g) [mol]"].iloc[11]
        last_number_global_h2 = pd.to_numeric(last_number_global_h2, errors='coerce')
        h2g[i, 0] = last_number_global_h2
        print("The H2(g) [mol] after 1Y is: ", last_number_global_h2)


The H2(g) [mol] after 1Y is:  90565881300.0
The H2(g) [mol] after 1Y is:  89459571300.0
The H2(g) [mol] after 1Y is:  87931321000.0
The H2(g) [mol] after 1Y is:  87556123700.0
The H2(g) [mol] after 1Y is:  87491448600.0
The H2(g) [mol] after 1Y is:  87441829900.0
The H2(g) [mol] after 1Y is:  87382420200.0
The H2(g) [mol] after 1Y is:  87361979500.0
The H2(g) [mol] after 1Y is:  87351470700.0
The H2(g) [mol] after 1Y is:  87344988700.0
The H2(g) [mol] after 1Y is:  87340551000.0
The H2(g) [mol] after 1Y is:  87337294400.0
The H2(g) [mol] after 1Y is:  87334788100.0
The H2(g) [mol] after 1Y is:  87332789900.0
The H2(g) [mol] after 1Y is:  87331153200.0
The H2(g) [mol] after 1Y is:  90565849800.0
The H2(g) [mol] after 1Y is:  88586545400.0
The H2(g) [mol] after 1Y is:  87512507700.0
The H2(g) [mol] after 1Y is:  87436295500.0
The H2(g) [mol] after 1Y is:  87402830600.0
The H2(g) [mol] after 1Y is:  87377262500.0
The H2(g) [mol] after 1Y is:  87346290400.0
The H2(g) [mol] after 1Y is:  87

In [10]:
print(h2aq)
print(h2saq)
print(h2g)

np.savetxt('./H2AQ_Base_2_1Y.txt', h2aq, fmt='%d', delimiter=', ')
np.savetxt('./H2SAQ_Base_2_1Y.txt', h2saq, fmt='%.10e', delimiter=', ')
np.savetxt('./H2G_Base_2_1Y.txt', h2g, fmt='%d', delimiter=', ')

[[0.00000000e+00]
 [7.57614533e+08]
 [7.48126395e+08]
 [7.34956453e+08]
 [7.31810493e+08]
 [7.31254801e+08]
 [7.30828470e+08]
 [7.30318010e+08]
 [7.30142386e+08]
 [7.30052098e+08]
 [7.29996408e+08]
 [7.29958286e+08]
 [7.29930312e+08]
 [7.29908783e+08]
 [7.29891621e+08]
 [7.29877566e+08]
 [7.57616630e+08]
 [7.40605614e+08]
 [7.31438003e+08]
 [7.30783174e+08]
 [7.30495635e+08]
 [7.30275946e+08]
 [7.30009843e+08]
 [7.29913948e+08]
 [7.29861764e+08]
 [7.29827639e+08]
 [7.29802927e+08]
 [7.29783827e+08]
 [7.29768416e+08]
 [7.29755595e+08]
 [7.29744680e+08]
 [7.46076300e+08]
 [7.36529611e+08]
 [7.32568836e+08]
 [7.31511263e+08]
 [7.31027465e+08]
 [7.30652580e+08]
 [7.30202378e+08]
 [7.30049522e+08]
 [7.29972734e+08]
 [7.29926640e+08]
 [7.29895962e+08]
 [7.29874087e+08]
 [7.29857732e+08]
 [7.29845049e+08]
 [7.29834931e+08]
 [7.84471924e+08]
 [7.69086558e+08]
 [7.47170894e+08]
 [7.39498803e+08]
 [7.39198014e+08]
 [7.38967021e+08]
 [7.38675677e+08]
 [7.38565034e+08]
 [7.38500790e+08]
 [7.384563